In [1]:
import math
import pandas as pd 
import torch 
from torch import nn
from d2l import torch as d2l

In [2]:
class PositionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, ):
        super().__init__()

    def forward(self, X):
        pass

SyntaxError: unexpected EOF while parsing (2473475076.py, line 5)

In [ ]:
class DotProductionAttention(nn.Module):
    def __init__(self, dropout, **kwargs) -> None:
        super(DotProductionAttention, self).__init__(**kwargs)
        self.dropout = dropout
    
    def forward(self, queries, keys, values, vaild_lens=None):
        d = queries.shape[-1]
        scores = torch.bmm(queries, keys.transpose(1,2)/math.sqrt(d))
        self.attention_weights = d2l.masked_soft()

In [ ]:
d2l.DotProductAttention

In [3]:
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
n1 = torch.ones((2,3,4))
n1[0]

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [4]:
ffn(n1)[0]

tensor([[-0.0823, -0.1024,  0.6236,  0.1950, -0.0718, -0.4328,  0.0861,  0.3967],
        [-0.0823, -0.1024,  0.6236,  0.1950, -0.0718, -0.4328,  0.0861,  0.3967],
        [-0.0823, -0.1024,  0.6236,  0.1950, -0.0718, -0.4328,  0.0861,  0.3967]],
       grad_fn=<SelectBackward0>)

In [31]:
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)
X = torch.tensor([[1, 2], [2,3], [4,5]], dtype = torch.float32)
print(ln(X))
print(bn(X))
print(X.shape)

tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>)
tensor([[-1.0690, -1.0690],
        [-0.2673, -0.2673],
        [ 1.3363,  1.3363]], grad_fn=<NativeBatchNormBackward0>)
torch.Size([3, 2])


In [51]:
ln = nn.LayerNorm([3,2])
bn = nn.BatchNorm1d(2)
X = torch.tensor([[[1, 2], [2,3], [4,5]],[[1, 2], [2,3], [4,5]]], dtype = torch.float32)
print(X.shape)
print(ln(X))
# print(bn(X))

torch.Size([2, 3, 2])
tensor([[[-1.3644, -0.6202],
         [-0.6202,  0.1240],
         [ 0.8682,  1.6124]],

        [[-1.3644, -0.6202],
         [-0.6202,  0.1240],
         [ 0.8682,  1.6124]]], grad_fn=<NativeLayerNormBackward0>)


In [49]:
ln = nn.LayerNorm(2)
bn = nn.BatchNorm2d()
X = torch.ones(2,2,3,4)
print(X.shape)
# print(ln(X))
print(bn(X))

TypeError: __init__() missing 1 required positional argument: 'num_features'

In [5]:
class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + (X))

In [6]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()
add_norm(torch.ones((2,3,4)), torch.ones((2,3,4))).shape

torch.Size([2, 3, 4])

In [7]:
class EncoderBLock(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout, use_bias=False, **kwargs):
        super(EncoderBLock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(key_size, query_size, value_size, num_hiddens, num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)
    
    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

